In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

In [ ]:
source.subarray.peek()

In [ ]:
print(source.subarray.tel)

In [ ]:
#filename = '/Users/vdk/MST_work/data_muons/mono_proton_simtel/proton_20deg_0deg_run2___cta-prod6-paranal-2147m-Paranal-nshow-3000-0-LSTs-1-MSTs-0-SSTs-NSCAT_5_CSCAT_300_EMIN_1_EMAX_200_after_muontag.simtel.zst'
#filename = '/Users/vdk/MST_work/data_muons/mono_muon_simtel/muon_20deg_0deg_run0___cta-prod6-paranal-2147m-Paranal-0-LSTs-1-MSTs-0-SSTs-NSCAT_1_CSCAT_8_EMIN_0.005_EMAX_1_FIXCHI_600_VIEWCONE_3_after_muontag.simtel.zst'
#filename = '/Users/vdk/sim_run_folder/test321.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)



for i,j in enumerate(event_iterator):
    if i == 1:
        telescope = 1
        event = j
        image_processor = ImageProcessor(source.subarray)
        muon_processor = MuonProcessor(source.subarray)
        calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
        calib(j)
        image_processor(j)
        muon_processor(j)
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[telescope].camera.geometry
        title="TrueImage run {} if {} fit {} width {}".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[telescope].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        for pix_id in range(0,1764):
            bx.plot(j.r0.tel[telescope].waveform[0][pix_id], label="pix {}".format(pix_id), drawstyle='steps')
        bx.set_ylabel("ADC counts")
        bx.set_xlabel("Samples")
        bx.set_title("Waveforms")
        plt.show()
        #plt.savefig(f"/Users/vdk/MST_ProtonImages_NewQualityCheck/TrueImage+Waveform_mstPROTON_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        break
        #if i == 150:
        #    break

plt.close()

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)
for i,j in enumerate(event_iterator):
    print(i, j.trigger.tels_with_trigger)
    

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)

In [ ]:
event = next(event_iterator)

In [ ]:
for i in range(len(event.trigger.tels_with_trigger)):
    print(event.trigger.tels_with_trigger[i])




In [ ]:
camgeom = source.subarray.tel[1].camera.geometry

In [ ]:
disp = CameraDisplay(camgeom)
disp.image = next(event_iterator).simulation.tel[3].true_image

In [ ]:
event = next(event_iterator)

In [ ]:
print(len(event.simulation.tel[10].true_image))

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

events = [copy.deepcopy(event) for event in source]


for i,j in enumerate(event_iterator):
    print(f"Telescope with data: {event.r1.tel.keys()} in the events number {i}")

In [ ]:
obstime = Time("2013-11-01T03:00")
location = EarthLocation.of_site("Roque de los Muchachos")

altaz = AltAz(location=location, obstime=obstime)

array_pointing = SkyCoord(
    alt=event.pointing.array_azimuth,
    az=event.pointing.array_altitude,
    frame=altaz,
)

print(array_pointing)

In [ ]:
filename = '/Users/vdk/sim_run_folder/corsika_theta_28.4_az_182.75_run2.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

events = [copy.deepcopy(event) for event in source]
events[4]

In [ ]:
events[12].trigger.tel[3]

In [ ]:
stereo_mask = []
for i in events:
    check_LST1 = 1 in i.trigger.tels_with_trigger
    check_LST2 = 2 in i.trigger.tels_with_trigger
    if check_LST1 and check_LST2:
        stereo_mask.append(True)
        print(True)
    else:
        stereo_mask.append(False)        
        print(False)

In [ ]:
stereo_events = []
for i,j in enumerate(events):
    if stereo_mask[i]:
        stereo_events.append(j)
        print(j.trigger.tels_with_trigger)


In [ ]:
#for i in stereo_events:
    #print(i.trigger.tels_with_trigger)
len(stereo_events)

In [ ]:
source.subarray.peek()

In [ ]:
telescope = 1
camgeom = source.subarray.tel[telescope].camera.geometry
disp = CameraDisplay(camgeom)
disp.image = events[12].simulation.tel[telescope].true_image #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()

In [ ]:
telescope = 3
camgeom = source.subarray.tel[telescope].camera.geometry
disp = CameraDisplay(camgeom)
disp.image = events[6].simulation.tel[telescope].true_image #.sum(axis=1)
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()

In [ ]:
stereo_events[2].trigger.tels_with_trigger

In [ ]:
filename = '/Users/vdk/sim_run_folder/test123.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

for i,j in enumerate(event_iterator):
        telescope = 1
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[telescope].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = bx)
        disp.image = j.simulation.tel[2].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #bx.set_ylabel("ADC counts")
        #bx.set_xlabel("")
        bx.set_title("LST2")
        #plt.show()
        plt.savefig(f"/Users/vdk/StereoProton_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        #if i == 150:
        #    break

plt.close()

In [ ]:
# pipeline to extract only stereo trigger for 2 LSTs
filename = '/Users/vdk/sim_run_folder/run1_nt2_mt2_at1.simtel.gz'
source = EventSource(filename, max_events=10000)
event_iterator = iter(source)

events = [copy.deepcopy(event) for event in source]

stereo_mask = []
for i in events:
    check_LST1 = 1 in i.trigger.tels_with_trigger
    check_LST2 = 2 in i.trigger.tels_with_trigger
    if check_LST1 and check_LST2:
        stereo_mask.append(True)
        #print(True)
    else:
        stereo_mask.append(False)        
        #print(False)

stereo_events = []
for i,j in enumerate(events):
    if stereo_mask[i]:
        stereo_events.append(j)
        #print(j.trigger.tels_with_trigger)

print("events = ", len(events))
print("stereo events = ", len(stereo_events))
source.subarray.peek()

In [ ]:
for i,j in enumerate(stereo_events):
        telescope = 1
        event = j
        fig, axes = plt.subplots(2, 1, sharex=False, gridspec_kw={'height_ratios': [1, 1]}, figsize=(9,12))
        fig.subplots_adjust(hspace=0.2)
        ax = axes[0]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = ax)
        disp.image = j.simulation.tel[telescope].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        bx = axes[1]
        camgeom = source.subarray.tel[1].camera.geometry
        title="TrueImage stereo trigger LST1".format(j.index.obs_id, j.index.event_id, round(j.muon.tel[telescope].efficiency.optical_efficiency, 3), round(j.muon.tel[telescope].efficiency.width.to_value(),3))
        disp = CameraDisplay(camgeom,title=title, ax = bx)
        disp.image = j.simulation.tel[2].true_image #.sum(axis=1)
        #disp.image = event.dl1.tel[1].image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        #bx.set_ylabel("ADC counts")
        #bx.set_xlabel("")
        bx.set_title("LST2")
        #plt.show()
        plt.savefig(f"/Users/vdk/StereoProtonNT2MT2_AT1_event{j.index.event_id}_obs{j.index.obs_id}_numer{i}")
        #plt.savefig(f"/Users/vdk/TrueImage+Waveform_mstMUON_event{j.index.event_id}_obs{j.index.obs_id}")
        #if i == 150:
        #    break

plt.close()

In [ ]:
for i in stereo_events:
    print(i.trigger.tels_with_trigger)

In [ ]:
source.subarray.tels

In [ ]:
# pipeline to extract only stereo trigger for 2 LSTs
all_events = []
for i in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/numtele2_maxtele2_arraytelescope1/corsika_theta_28.4_az_182.75_run{i}.simtel.gz'
    source = EventSource(filename, max_events=10000)
    events = [copy.deepcopy(event) for event in source]
    all_events.append(events)

#stereo_mask = []
#for i in events:
#    check_LST1 = 1 in i.trigger.tels_with_trigger
#    check_LST2 = 2 in i.trigger.tels_with_trigger
#    if check_LST1 and check_LST2:
#        stereo_mask.append(True)
#        #print(True)
#    else:
#        stereo_mask.append(False)        
#        #print(False)

#stereo_events = []
#for i,j in enumerate(events):
#    if stereo_mask[i]:
#        stereo_events.append(j)
#        #print(j.trigger.tels_with_trigger)

#print("events = ", len(events))
#print("stereo events = ", len(stereo_events))
#source.subarray.peek()



In [ ]:
full_stereo_events = []
count = 0
for j in range(1,40):
    for i in all_events[j]:
        count += 1
        if len(i.trigger.tels_with_trigger) > 1:
            full_stereo_events.append(i)

In [ ]:
len(full_stereo_events)

In [ ]:
count

In [ ]:
2745/11191

In [ ]:
checker = 0
efficiency_container = []
for k in range(1,40):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=9000)
    event_iterator = iter(source)
    for i,j in enumerate(event_iterator):
        checker += 1
        if len(j.trigger.tels_with_trigger) > 1:
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(
                image_extractor_type="GlobalPeakWindowSum",
                subarray = source.subarray)
            calib(j)
            image_processor(j)
            muon_processor(j)
            for m in j.muon.tel.keys():
                efficiency_container.append(j.muon.tel[m].efficiency)
            efficiency_container.append(j)
        
        
            
            
        

In [ ]:
filename = '/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run1.simtel.gz'
source = EventSource(filename, max_events=9000)
event_iterator = iter(source)



In [ ]:
event = next(event_iterator)

In [ ]:
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray)
calib(event)
image_processor(event)
muon_processor(event)

In [ ]:
for i in event.muon.tel.keys():
    print(event.muon.tel[i].efficiency)

In [ ]:
test[0].width